In [1]:
import pandas as pd
import re
from konlpy.tag import Okt
# 텍스트 데이터를 머신러닝 알고리즘이 처리할 수 있는 수치 벡터로 변환
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import warnings
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import re
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("C:/Users/82105/Downloads/divorce_df.csv", encoding="utf-8-sig")

In [5]:
print("=== 이혼 관련 문서만 토픽 모델링 시작 ===")

print(f"📊 데이터 분할:")
print(f"  • 이혼 관련 문서: {len(df)}개")

=== 이혼 관련 문서만 토픽 모델링 시작 ===
📊 데이터 분할:
  • 이혼 관련 문서: 741개


In [3]:
print("=== 데이터 전처리: null 값 제거 ===")
print(f"전처리 전 데이터 수: {len(df)}")

# null 값이 있는 행 제거
df_clean = df.dropna(subset=['input']).copy()
print(f"null 값 제거 후 데이터 수: {len(df_clean)}")
print()

=== 데이터 전처리: null 값 제거 ===
전처리 전 데이터 수: 741
null 값 제거 후 데이터 수: 711



# 1단계 : KoNLPy를 사용한 정교한 전처리

In [4]:
MINIMAL_STOPWORDS = [
    '것', '수', '때', '등', '들', '더', '이', '그', '저', '나', 
    '우리', '같', '또', '만', '년', '월', '일', '하다', '있다', '되다',

     '가능하다', '가능', '가다', '되다', '하다', '있다', '없다', '않다', '된다', '한다',
    '어떻게', '어떤', '무엇', '언제', '어디서', '왜', '누가', '얼마', '몇',
    '알고', '싶다', '궁금하다', '문의', '질문', '답변', '설명', '이해',
    '과정', '절차', '이후', '다음', '먼저', '그리고', '그러나', '하지만', '그래서', '제자','제호','제조',

     '없', '있', '하', '되', '않', 
    '나', '우리', '너', '당신', '같', '또', '것', '때', '등', 
    '때문', '정도', '사실', '생각', '경우', '문제', '방법', '상황', '내용', '결과', '사람',

    '해야', '하면', '경우', '때는', '어느', '무슨', '어디', '누구' , ' 가지다' , '가지''하나요', '위해', '이혼',
     '조건', '대한', '관련', '따르다', '판단', '인정', '적용', 
    '효력', '성립', '발생', '요건', '증명', '근거', '주장'
]
# 중복 제거 후 사용
MINIMAL_STOPWORDS = list(set(MINIMAL_STOPWORDS))

# # 법률 전문용어는 무조건 포함
LEGAL_KEYWORDS = [ '위자료', '재산분할', '양육권', '친권', '면접교섭', 
    '협의이혼', '청구', '배상', '손해', '책임',
    '차용금', '반환', '취소', '원상회복', '사해행위', '채권자', '배우자', '이혼사유',
     '이혼', '사유', 
    '혼인', '금전거래', '청구권',  '액수', '정해지', '부적법',
    '혼인파탄', '파탄', '분할', '양육비', '면접',
    '교섭', '협의', '조정신청', '손해배상', '부부', '배우자', '당사자', '사람', '개인', '상대방', 
]

In [5]:
def preprocess_text(text):  # 전처리 함수 
    """
    한국어 텍스트를 입력받아 전처리하는 함수:
    1. '제3자' 형태의 단어를 임시 토큰으로 보호/복원하여 숫자를 보존.
    2. 불필요한 한글 이외의 문자를 제거.
    3. 형태소 분석 및 어간 추출.
    4. 불용어 및 1글자 단어 제거.
    """
    # 1단계: 한글, 공백을 제외한 모든 문자 제거
    # re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', text)는 한글과 공백만 남기고 나머지는 지우라는 의미
    # 제숫자 + 한글 글자 + 선택적 조사까지 포함
    # 딕셔너리를 함수 내부에 선언하여 매 호출마다 초기화 (핵심 수정 사항)
    protected_matches = {}

    def protect_term(match):
        # 찾은 문자열(예: '제3자')을 고유한 토큰(예: '###TOKEN0###')으로 매핑
        token = f"###TOKEN{len(protected_matches)}###"
        protected_matches[token] = match.group(0)
        return match.group(0) # 일단 원본 단어 그대로 둔 채로 형태소 분석 진행
    def strip_josa(text):
    # 한글 명사 뒤 “의”, “가”, “을” 등 제거
        return re.sub(r'([가-힣]+)(의|가|를|은|는|과|와)$', r'\1', text)
    

    # 1단계: '제3자' 형태의 단어를 임시 토큰으로 치환하여 보호
    re.sub(r'(제\d+[가-힣]+)', protect_term, text)

    # 2단계: 한글, 공백, 보호 토큰의 문자(#)만 남기고 제거
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣# ]', '', text)

    # 2단계: Okt 형태소 분석기를 이용한 토큰화 및 어간 추출(Stemming)
    # okt.pos(text, stem=True)는 문장을 (단어, 품사) 형태로 나누고, '하다', '먹다'처럼 원형으로 만들어줍니다.
    # 예: "먹었었고" -> ('먹다', 'Verb')
    # Okt 형태소 분석기 객체 생성
    okt = Okt() 
    word_tokens = okt.pos(text, stem=True)

    # 3단계: 불용어 제거
    # 형태소 분석 후 의미 있는 단어만 추출하는 필터링 과정
    # 의미를 가지는 명사, 동사, 형용사, 부사 중에서 1글자 이상인 단어만 추출합니다.
    # 품사 태그가 'Josa'(조사), 'Eomi'(어미), 'Punctuation'(구두점) 등인 단어들을 제거합니다.
    meaningful_words = []
    for word, pos in word_tokens:
        if word in LEGAL_KEYWORDS:
            meaningful_words.append(word)
        elif pos in ['Noun','Verb'] and len(word) > 1:
            token = strip_josa(word)
            if token not in MINIMAL_STOPWORDS:
                meaningful_words.append(token)
     # 6. 보호된 단어 강제 포함 및 복원 (핵심)
    # 보호 목록에 있던 단어들을 강제로 최종 리스트에 추가합니다.
    # (이미 리스트에 분해되어 들어갔을 수 있지만, 완벽한 복원을 위해 강제 추가)
    for original_term in protected_matches.values():
        # '제3자' 자체를 하나의 단어로 명시적으로 추가
        meaningful_words.append(original_term)
        
    # 7. 중복 제거 및 최종 문자열 반환
    final_words = list(dict.fromkeys(meaningful_words))
    # 최종적으로 공백으로 연결된 문자열을 반환합니다. 
    # 모델에 따라 리스트 형태(meaningful_words)를 그대로 사용할 수도 있습니다.
    # 텍스트에서 분석에 의미 있는 핵심 단어들만 남긴 리스트 생성
    return ' '.join(final_words)

In [6]:
# --- 4. 각 목적에 맞게 전처리 컬럼 생성 ---
# 질문 의도 파악 모델용: 'input' 컬럼만 전처리
# 'input' 컬럼에 전처리 함수를 적용하여 새로운 'input_processed' 컬럼 생성
df_clean['input_processed'] = df_clean['input'].apply(preprocess_text)
df_clean['combined_processed'] = df_clean['text_combined'].apply(preprocess_text)

# 결과 비교를 위해 원본과 처리된 결과를 나란히 출력
print("\n=== 전처리 전/후 비교 ===")
for i in range(5):
    print(f"원본 [{i}]: {df_clean['input'].iloc[i]}")
    print(f"결과 [{i}]: {df_clean['input_processed'].iloc[i]}\n")


# 전처리된 데이터가 포함된 데이터프레임 확인
print("=== 각 목적에 맞게 생성된 전처리 컬럼들 ===")
print(df_clean[['input', 'input_processed']].head())


=== 전처리 전/후 비교 ===
원본 [0]: 혼인 중 발생한 금전거래와 관련하여 차용금 반환 청구권이 인정되는 조건은 무엇인가요?
결과 [0]: 혼인 금전 거래 차용 반환 청구권

원본 [1]: 제3자가 부부의 일방과 부정행위를 할 경우 어떤 법적 책임을 지게 되나요?
결과 [1]: 부부 일방 부정행위 법적 책임 지게 제3자가

원본 [2]: 민법 제840조 제1호에서 규정한 재판상 이혼사유 중 부정한 행위의 의미는 무엇인가요?
결과 [2]: 민법 규정 재판 이혼 사유 부정 행위 의미 제840조 제1호에서

원본 [3]: 배우자가 있는 사람과 부정행위를 한 경우 위자료의 액수는 어떻게 정해지나요?
결과 [3]: 배우자 부정행위 위자료 액수 정해지다

원본 [4]: 사해행위 취소 및 원상회복청구 소송에서 다른 채권자의 청구가 부적법해지는 상황은 어떤 경우인가요?
결과 [4]: 사해행위 취소 원상회복 청구 소송 다른 채권자 부적 법해

=== 각 목적에 맞게 생성된 전처리 컬럼들 ===
                                               input  \
0    혼인 중 발생한 금전거래와 관련하여 차용금 반환 청구권이 인정되는 조건은 무엇인가요?   
1          제3자가 부부의 일방과 부정행위를 할 경우 어떤 법적 책임을 지게 되나요?   
2   민법 제840조 제1호에서 규정한 재판상 이혼사유 중 부정한 행위의 의미는 무엇인가요?   
3         배우자가 있는 사람과 부정행위를 한 경우 위자료의 액수는 어떻게 정해지나요?   
4  사해행위 취소 및 원상회복청구 소송에서 다른 채권자의 청구가 부적법해지는 상황은 어...   

                       input_processed  
0                   혼인 금전 거래 차용 반환 청구권  
1             부부 일방 부정행위 법적 책임 지게 제3자가  
2  민법 규정 재판 이혼 사유 부정 행위 의미 제840조 제1호

# 2단계: 최적화된 TF-IDF 벡터화

In [7]:
# TF-IDF Vectorizer 설정
vectorizer = TfidfVectorizer(
    max_features=1000,      # 상위 빈도 1000개 단어만 사용
    min_df=3,
    max_df=0.1,           
    lowercase=False,         # 소문자 변환 x
    ngram_range=(1, 2),     # 1-gram, 2-gram 모두 사용
    sublinear_tf=True,            # TF 값에 로그 스케일 적용 (성능 향상)
    token_pattern=r'[가-힣]{2,}', # 한글 2글자 이상
)

# TF-IDF 행렬 생성
tfidf_matrix = vectorizer.fit_transform(df_clean['input_processed'])  # X는 (문서 수 × 단어 수) 크기의 희소 행렬

print(f"TF-IDF 행렬 크기: {tfidf_matrix.shape}")
print(f"문서 수: {tfidf_matrix.shape[0]}")
print(f"특성(단어) 수: {tfidf_matrix.shape[1]}")

# 특성 이름(단어들) 확인
# vectorizer.get_feature_names_out()로 어떤 단어가 벡터에 들어갔는지 확인 가능
feature_names = vectorizer.get_feature_names_out()
print(f"상위 20개 특성: {feature_names[:20]}")

TF-IDF 행렬 크기: (711, 411)
문서 수: 711
특성(단어) 수: 411
상위 20개 특성: ['가지다' '가하다' '가하다 법적' '가하다 불법행위' '가하다 손해배상' '간주' '간주 자가' '간통' '개입'
 '개입 손해배상' '개입 파탄' '개입 혼인' '거래' '겪다' '결정' '결정 고려' '결정 기준' '계산' '계약' '고려']


# 3단계: 토픽 개수 최적화
# 최적의 토픽 개수를 찾기 위해 여러 지표를 사용

# 더 정확한 토픽 모델링을 위해 Gensim 라이브러리를 사용
# 고급 기법: Gensim LDA + Coherence Score

In [8]:
from gensim.models.phrases import Phrases, Phraser

# 텍스트를 토큰 리스트로 변환
texts = [text.split() for text in df_clean['input_processed']]

# 1. Bi-gram 모델 생성
phrases = Phrases(texts, min_count=5, threshold=10) # min_count, threshold는 데이터에 맞게 조정
bigram = Phraser(phrases)

# 2. Bi-gram 적용
texts_bigram = [bigram[doc] for doc in texts]

# (선택) Tri-gram까지 적용하고 싶다면 한 번 더 수행
# phrases_tri = Phrases(texts_bigram, min_count=5, threshold=10)
# trigram = Phraser(phrases_tri)
# texts_trigram = [trigram[doc] for doc in texts_bigram]

# 이제 texts_bigram (또는 texts_trigram)을 사용하여 Dictionary와 Corpus를 만드세요.
# dictionary = Dictionary(texts_bigram)
# corpus = [dictionary.doc2bow(text) for text in texts_bigram]

In [9]:
# Dictionary 생성
dictionary = Dictionary(texts_bigram)
dictionary.filter_extremes(no_below=2, no_above=0.8)

# Corpus 생성  
corpus = [dictionary.doc2bow(text) for text in texts_bigram]

# 최적 토픽 수 찾기 (Coherence Score 사용)
coherence_scores = []
models_dict = {}
topic_range = range(8, 10)

for num_topics in topic_range:
    lda_model = LdaModel(
        corpus=corpus,
        id2word=dictionary, 
        num_topics=num_topics,
        random_state=42,
        passes=30,
        alpha='auto',        # 자동 조정
        per_word_topics=True
        
    )
    
    coherence_model = CoherenceModel(
        model=lda_model, 
        texts=texts_bigram, 
        dictionary=dictionary, 
        coherence='c_v'
    )
    
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append(coherence_score)
    models_dict[num_topics] = lda_model
    print(f"토픽 수 {num_topics}: Coherence = {coherence_score:.4f}")

# 상위 3개 후보 선정
top_3_indices = np.argsort(coherence_scores)[-3:]
top_3_topics = [topic_range[i] for i in top_3_indices]
print(f"\nCoherence 상위 3개: {top_3_topics}")


# 각 후보의 실제 토픽 내용 확인
for num_topics in top_3_topics:
    print(f"\n=== {num_topics}개 토픽 결과 ===")
    model = models_dict[num_topics]
    
    for topic_id in range(num_topics):
        terms = model.show_topic(topic_id, topn=8)
        words = [term[0] for term in terms]
        print(f"토픽 {topic_id}: {', '.join(words)}")

토픽 수 8: Coherence = 0.3599
토픽 수 9: Coherence = 0.3655

Coherence 상위 3개: [8, 9]

=== 8개 토픽 결과 ===
토픽 0: 부부, 제3자가, 불법행위, 부정행위, 책임, 생활_침해, 행위, 타인
토픽 1: 재산_분할, 청구, 이혼, 양육비, 의무, 부부, 지급, 이루어지다
토픽 2: 법적, 부정행위, 책임, 배우자, 제3자가, 부부_일방, 지게, 손해배상
토픽 3: 기준, 부정행위, 청구, 위자료, 손해배상, 산정, 결정, 위자료_액수
토픽 4: 혼인_관계, 파탄, 공동, 의무, 부부, 생활, 기준, 유지
토픽 5: 부정행위, 책임, 제3자가, 손해배상, 부부_일방, 배우자, 불법행위, 재산_분할
토픽 6: 손해배상, 부정행위, 청구, 배우자, 정신, 정신_고통, 손해, 범위
토픽 7: 부정_행위, 범위, 법률, 이혼, 민법, 배우자, 이혼소송, 평

=== 9개 토픽 결과 ===
토픽 0: 부부, 책임, 제3자가, 부정행위, 손해배상, 대해, 불법행위, 타인
토픽 1: 이혼, 재산_분할, 청구, 양육비, 의무, 지급, 법적, 이루어지다
토픽 2: 법적, 부정행위, 책임, 다른, 부부, 제3자는, 파탄_초래, 한쪽
토픽 3: 기준, 부정행위, 위자료_액수, 산정, 결정, 사실혼_관계, 손해배상, 청구
토픽 4: 부부, 혼인_관계, 파탄, 불법행위, 제3자가, 책임, 행위, 제3자의
토픽 5: 부정행위, 제3자가, 책임, 부부_일방, 배우자, 손해배상, 법적, 불법행위
토픽 6: 청구, 손해배상, 부정행위, 배우자, 정신_고통, 위자료, 정신, 결정
토픽 7: 부정_행위, 민법, 규정, 간통, 권리, 범위, 제840조, 배우자
토픽 8: 범위, 손해, 정신, 배상, 법률, 불법행위, 청구, 부정행위


In [14]:
for num_topics in top_3_topics:
    print(f"\n=== {num_topics}개 토픽 결과 ===")
    model = models_dict[num_topics]
    
    for topic_id in range(num_topics):
        terms = model.show_topic(topic_id, topn=8)
        topic_str = ', '.join([f"{word}({weight:.4f})" for word, weight in terms])
        print(f"토픽 {topic_id}: {topic_str}")


=== 8개 토픽 결과 ===
토픽 0: 부부(0.1298), 제3자가(0.0844), 불법행위(0.0526), 부정행위(0.0511), 책임(0.0484), 생활_침해(0.0454), 행위(0.0372), 타인(0.0365)
토픽 1: 재산_분할(0.0883), 청구(0.0731), 이혼(0.0708), 양육비(0.0546), 의무(0.0534), 부부(0.0380), 지급(0.0378), 이루어지다(0.0345)
토픽 2: 법적(0.1050), 부정행위(0.0966), 책임(0.0859), 배우자(0.0613), 제3자가(0.0552), 부부_일방(0.0308), 지게(0.0302), 손해배상(0.0241)
토픽 3: 기준(0.0945), 부정행위(0.0918), 청구(0.0906), 위자료(0.0728), 손해배상(0.0569), 산정(0.0399), 결정(0.0365), 위자료_액수(0.0349)
토픽 4: 혼인_관계(0.1025), 파탄(0.0848), 공동(0.0640), 의무(0.0410), 부부(0.0402), 생활(0.0373), 기준(0.0299), 유지(0.0246)
토픽 5: 부정행위(0.1208), 책임(0.1058), 제3자가(0.0998), 손해배상(0.0870), 부부_일방(0.0712), 배우자(0.0522), 불법행위(0.0429), 재산_분할(0.0315)
토픽 6: 손해배상(0.1359), 부정행위(0.1300), 청구(0.1235), 배우자(0.0790), 정신(0.0590), 정신_고통(0.0494), 손해(0.0473), 범위(0.0302)
토픽 7: 부정_행위(0.0839), 범위(0.0565), 법률(0.0553), 이혼(0.0375), 민법(0.0346), 배우자(0.0324), 이혼소송(0.0307), 평(0.0297)

=== 9개 토픽 결과 ===
토픽 0: 부부(0.0798), 책임(0.0732), 제3자가(0.0684), 부정행위(0.0649), 손해배상(0.0548), 대해(0.0366), 불법행위(0

# 4단계: 최종 토픽 라벨 할당

In [10]:
# 8개 토픽 모델 선택
optimal_num_topics = 9
best_model = models_dict[optimal_num_topics]  # coherence 결과에서 9개 토픽 모델

print(f"선택된 토픽 수: {optimal_num_topics}")

선택된 토픽 수: 9


In [12]:
df_clean2 = df_clean.copy()

In [13]:
# 각 문서가 어떤 토픽에 속하는지 결정하는 단계
def assign_document_topics(model, corpus):
    topic_assignments = []
    topic_probabilities = []
    
    for doc_bow in corpus:
        doc_topics = model.get_document_topics(doc_bow)
        
        if doc_topics:
            # 가장 높은 확률의 토픽 찾기
            dominant_topic = max(doc_topics, key=lambda x: x[1])
            topic_assignments.append(dominant_topic[0])      # 토픽 번호
            topic_probabilities.append(dominant_topic[1])    # 확률
        else:
            topic_assignments.append(-1)
            topic_probabilities.append(0.0)
    
    return topic_assignments, topic_probabilities

# 실행
topic_ids, topic_probs = assign_document_topics(best_model, corpus)

# DataFrame에 추가
df_clean2['topic_label'] = topic_ids
df_clean2['topic_probability'] = topic_probs

In [59]:
topic_labels = {
    0: "제3자 개입과 공동 불법행위 책임",      
    1: "이혼 시 금전 문제 (재산분할 및 양육비)",     
    2: "혼인 파탄의 원인과 법적 책임",       
    3: "위자료 액수 산정 기준 및 결정 요소",   
    4: "제3자 개입으로 인한 혼인 관계 파탄",     
    5: "유책배우자_제3자의 연대 책임_손해배상",     
    6: "정신적 고통에 대한 위자료 청구_액수",    
    7: "부정행위의 법률적 근거 (민법 제840조)",
    8: "손해배상의 법적 인정 범위"
    
}

# DataFrame에 토픽 이름 추가
df_clean2['topic_name'] = df_clean2['topic_label'].map(topic_labels)

In [60]:
# 1. 토픽별 대표 문서 확인
def show_topic_examples(df, topic_label, n_examples=3):
    topic_docs = df[df['topic_label'] == topic_id].sort_values('topic_probability', ascending=False)
    
    print(f"\n=== 토픽 {topic_id} 대표 문서 ===")
    for i, (idx, row) in enumerate(topic_docs.head(n_examples).iterrows()):
        print(f"{i+1}. (확률: {row['topic_probability']:.3f})")
        print(f"   {row['input'][:100]}...")

# 모든 토픽 확인
for topic_id in range(9):
    show_topic_examples(df_clean2, topic_id, n_examples=2)

# 2. 확률 분포 확인
print("토픽 확률 통계:")
print(df_clean2['topic_probability'].describe())

# 3. 애매한 할당 확인 (확률 < 0.3)
low_confidence = df_clean2[df_clean2['topic_probability'] < 0.3]
print(f"애매한 할당: {len(low_confidence)}개 ({len(low_confidence)/len(df_clean2)*100:.1f}%)")



=== 토픽 0 대표 문서 ===
1. (확률: 0.940)
   제3자가 부부 관계에 개입하여 혼인 생활을 방해한 경우 이는 불법 행위로 간주될 수 있는 조건은 무엇인가요?...
2. (확률: 0.940)
   부부공동생활을 방해하는 부정행위에 대해 제3자가 불법행위로 인한 손해배상 책임을 지는 경우는 무엇인가요?...

=== 토픽 1 대표 문서 ===
1. (확률: 0.941)
   양육비를 합의서에서 재산 이전이나 금전 지급으로 약정하는 경우 어떻게 해석되나요?...
2. (확률: 0.926)
   약정금 지급의무와 관련된 분쟁에서 양육비 채권과의 상계가 인정될 수 있는 조건은 무엇인가요?...

=== 토픽 2 대표 문서 ===
1. (확률: 0.924)
   부동산 매각대금의 지급과 관련하여 상속인들 간의 분쟁이 있을 때, 이를 해결하는 기본적인 법적 절차는 무엇인가요?...
2. (확률: 0.924)
   다른 사람과 부정행위를 하여 부부공동생활을 침해한 제3자는 어떤 법적 책임이 있나요?...

=== 토픽 3 대표 문서 ===
1. (확률: 0.964)
   '혼인관계 해소에 따른 재산분할이 확정되었음을 전제로 채무의 존부 확인 및 불이행을 이유로 한 손해배상 청구'의 소송이 계속 중인 경우 본소 청구와 반소 청구의 이유는 어떻게 되나...
2. (확률: 0.948)
   부부의 단독 명의로 취득한 부동산의 소유권 판단에서 특유재산의 추정은 어떻게 이루어지며, 이를 번복하기 위한 증명책임은 누구에게 있나요?...

=== 토픽 4 대표 문서 ===
1. (확률: 0.950)
   부부가 장기간 별거하여 부부공동생활이 파탄된 경우 제3자가 부부 일방과 성적인 행위를 할 때 법률적으로 어떻게 평가하나요?...
2. (확률: 0.939)
   제3자가 부부 관계에 개입하여 법적인 문제가 되는 행위를 했을 때, 어떤 사유로 불법행위가 성립하나요?...

=== 토픽 5 대표 문서 ===
1. (확률: 0.949)
   제3자가 부부의 일방과 부정행위

In [61]:
# 이 2개는 어떤 문서인지 확인해보면 좋을 것 같아요
low_confidence = df_clean2[df_clean2['topic_probability'] < 0.3]
print("애매한 할당 문서들:")
for idx, row in low_confidence.iterrows():
    print(f"- {row['input'][:100]}...")

애매한 할당 문서들:


In [62]:
# 모든 토픽 이름 출력해서 확인
print("=== 전체 토픽 이름 목록 ===")
all_topics = df_clean2['topic_name'].value_counts()
for topic_name, count in all_topics.items():
    print(f"{topic_name}: {count}개")

=== 전체 토픽 이름 목록 ===
정신적 고통에 대한 위자료 청구_액수: 197개
유책배우자_제3자의 연대 책임_손해배상: 147개
이혼 시 금전 문제 (재산분할 및 양육비): 91개
위자료 액수 산정 기준 및 결정 요소: 75개
손해배상의 법적 인정 범위: 55개
제3자 개입과 공동 불법행위 책임: 50개
제3자 개입으로 인한 혼인 관계 파탄: 44개
혼인 파탄의 원인과 법적 책임: 30개
부정행위의 법률적 근거 (민법 제840조): 20개


In [64]:
# 상위 카테고리별 포함 토픽 이름 리스트
category_map = {
    "위자료_관련": [
        "정신적 고통에 대한 위자료 청구_액수",
        "위자료 액수 산정 기준 및 결정 요소",
        "손해배상의 법적 인정 범위",
        "유책배우자_제3자의 연대 책임_손해배상",
        "이혼 시 금전 문제 (재산분할 및 양육비)"
    ],
    "불법행위_책임": ["제3자 개입과 공동 불법행위 책임",
                    "제3자 개입으로 인한 혼인 관계 파탄",
                   "혼인 파탄의 원인과 법적 책임",
                   "부정행위의 법률적 근거 (민법 제840조)"]
}


In [66]:
# topic_name → 상위 카테고리 매핑 함수
def map_to_category(topic_name):
    for category, names in category_map.items():
        if topic_name in names:
            return category
    return "기타"

# 새로운 컬럼 생성
df_clean2['super_category'] = df_clean2['topic_name'].apply(map_to_category)

In [69]:
# 위자료_관련 문서 수
alimony_broad = df_clean2[df_clean2['super_category'] == "위자료_관련"]
print(f"폭넓은 위자료 관련 사건: {len(alimony_broad)}개")

# 토픽별 건수
print(alimony_broad['topic_name'].value_counts())

폭넓은 위자료 관련 사건: 565개
topic_name
정신적 고통에 대한 위자료 청구_액수       197
유책배우자_제3자의 연대 책임_손해배상      147
이혼 시 금전 문제 (재산분할 및 양육비)     91
위자료 액수 산정 기준 및 결정 요소        75
손해배상의 법적 인정 범위              55
Name: count, dtype: int64


In [70]:
# 최종 결과 저장
final_columns = ['input', 'output', 'input_processed', 'is_divorce', 'topic_label',  'super_category', 'topic_name', 'topic_probability', 'announce_date']

final_df = df_clean2[final_columns].copy()
final_df.to_csv("C:/Users/82105/Downloads/divorce_topic_labeled_input.csv", index=False, encoding='utf-8-sig')
print("최종 결과 저장 완료!")

최종 결과 저장 완료!
